<center>
<h4>Diplomatura AACSyA 2018 - FaMAF - UNC</h4>
<h3>Aprendizaje Supervisado</h3>
<hr>
<h4>Por David Gonzalez <leonardo.david.gonzalez@gmail.com> y Facundo Díaz Cobos <facundo.diaz.cobos@gmail.com></h4>
</center>
<br/>

# Aprendizaje Supervisado: Laboratorio 1

## Ejercicio 1: SVM con Kernel y Regularización

En este ejercicio se trabajará con datos que esconden un patrón. Los datos mezclan columnas con valores enteros, flotantes y booleanos.

Debe analizar los datos de entrenamiento para encontrar el patrón, y luego entrenar una SVM que explique a la perfección esos datos (accuracy 1.0 en train). Por último, deberá evaluar la SVM en los datos de test, y modificar los parámetros de regularización hasta encontrar una configuración que dé una accuracy mayor a 0.9.

In [ ]:
# cargar datos de entrenamiento y evaluación
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
train = pd.read_csv('datasets/lab1/ej1_train.csv')

X_train = train.iloc[:, :-1]
Y_train = train['TARGET']

test = pd.read_csv('datasets/lab1/ej1_test.csv')
X_test = test.iloc[:, :-1]
Y_test = test['TARGET']


from sklearn.decomposition import PCA
from sklearn.svm import SVC

### Ejercicio 1a:

Primero imprima una descripción básica de los datos.

Luego, calcule y grafique un mapa de calor con la correlación entre los distintos features.

**Preguntas:**
- ¿Qué observa?
- ¿Qué conclusiones puede extraer?

Por último, reduzca la dimensionalidad de los datos a dos dimensiones y grafique los puntos.

**Preguntas:**
- ¿Qué observa?
- ¿Qué conclusiones puede extraer?


In [ ]:
train.describe()


In [ ]:
# vemos la correlacion entre los features
train_corr = train.corr()
train_corr

In [ ]:
# graficamos la correlacion basado en el grafico de calor del problema del fraude de las tarjeras de crédito
plt.figure(figsize=(15,10))
seaborn.heatmap(train_corr, cmap="YlGnBu") # Displaying the Heatmap
seaborn.set(font_scale=2,style='white')

plt.title('Heatmap correlation')
plt.show()

### Correlacion:

A simple vista podemos obvserbar que los primeros 9 features estal altamente correlacionados.
por lo que debemos reducir la dimensionalidad en esos features.

In [ ]:
# Vamos a Aplicar PCA de la libreria de sklearn. 
# http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

pca = PCA(n_components=2)
pca.fit(X_train)
X_train_reduced = pca.transform(X_train)
X_test_reduced = pca.transform(X_test)



In [ ]:
X_train_reduced.shape

In [ ]:
X_train_reduced = pd.DataFrame(data=X_train_reduced, index=X_train_reduced[0:,0])

train_corr_reduced = X_train_reduced.corr()
train_corr_reduced

In [ ]:
# Graficamos las nuevas correlaciones

plt.figure(figsize=(15,10))
seaborn.heatmap(train_corr_reduced, cmap="YlGnBu") # Displaying the Heatmap
seaborn.set(font_scale=2,style='white')

plt.title('Heatmap correlation')
plt.show()

### Ejercico 1b:

Primero calcule la accuracy sobre train para una SVM con kernel lineal.

In [ ]:
from sklearn.metrics import accuracy_score
clf = SVC(random_state=0, kernel="linear")

clf.fit(X_train_reduced, Y_train)
Y_pred = clf.predict(X_train_reduced)

print('Accuracy en train:\t', accuracy_score(Y_train, Y_pred))

## **Preguntas:**
- ¿Cuál es la accuracy?
  Accuracy es la relacion entre los resultados predichos con los reales, asi pues si un clasificador 
    tiene una accuracy de 1 significa que clasifico correctamente el 100% de los casos de prueba,
    pero no siempre es una buena medida, ya que le da el mismo valor a falsos positivos y a falsos negativos,
    siendo esto un problema en determinados casos.
    (como el problema que vimos en clases del detector de cancer de mamas)
    
    En este caso arrojó una accuracy de **0.6225** usando PCA con dos componentes, usando todos los componentes **0.72**
  
- ¿Qué problema tiene el modelo? ¿De sesgo (bias) o de varianza?
  El modelo sufre de problemas de sesgo

----
### Modifique la SVM para lograr una accuracy en train de 1.0...
----

In [ ]:
from sklearn.metrics import accuracy_score
clf = SVC(random_state=0, kernel="rbf", C=2 )
# Aqui cambiamos de kernel, se podria haber intentado alguna transformacion como por ejemplo
# elevar features al cuadrado, y usar un kernel lineal

clf.fit(X_train_reduced, Y_train)
Y_pred = clf.predict(X_train_reduced)
print('Accuracy en train:\t', accuracy_score(Y_train, Y_pred))



### Ejercicio 1c:

Evalúe la SVM del ítem anterior sobre los datos de test.

In [ ]:
Y_pred = clf.predict(X_train_reduced)
print('Accuracy en train:\t', accuracy_score(Y_train, Y_pred))

Y_pred = clf.predict(X_test_reduced)
print('Accuracy en test:\t', accuracy_score(Y_test, Y_pred))

aca vemos claramente como se sobreentreno para los datos de test, resultando en un pesimo modelo

Modifique parámetros de la SVM para lograr una accuracy en test mayor a 0.9.
La accuracy en train puede ser menos de 1.0.

Puede buscar los valores apropiados a mano o usar grid search.

In [12]:
from sklearn.metrics import accuracy_score
clf = SVC(random_state=0, kernel="linear")

def square(x):
    return x*10

X_train_2 = X_train_reduced.apply(square)
Y_train_2 = Y_train.apply(square)



In [ ]:
import seaborn as sns
sns.set(style="ticks")

# Load the example dataset for Anscombe's quartet
# df = sns.load_dataset("anscombe")

# Show the results of a linear regression within each dataset
sns.lmplot(x=X_train_reduced col="dataset",
           col_wrap=2, ci=None, palette="muted", height=4,
           scatter_kws={"s": 50, "alpha": 1})

In [13]:
clf.fit(X_train_2, Y_train_2)



Y_pred = clf.predict(X_train_reduced)
Y_pred2 = clf.predict(X_test_reduced)
print('Accuracy en train:\t', accuracy_score(Y_train, Y_pred))
print('Accuracy en test:\t', accuracy_score(Y_test, Y_pred2))

KeyboardInterrupt: 

**Preguntas:**
- ¿Cuál es la accuracy en test?
- ¿Qué problema tiene el modelo? ¿De sesgo (bias) o de varianza?


In [19]:
Y_pred = clf.predict(X_train)
print('Accuracy en train:\t', accuracy_score(Y_train, Y_pred))
Y_pred = clf.predict(X_test)
print('Accuracy en test:\t', accuracy_score(Y_test, Y_pred))

ValueError: X.shape[1] = 17 should be equal to 2, the number of features at training time

## Ejercicio 2: Revisitando el Titanic

Este ejercicio se basa en el siguiente análisis de datos visto en clase:

- https://www.kaggle.com/zlatankr/titanic-random-forest-82-78/notebook

Aquí, se logra entrenar una Random Forest con un out-of-bag score de 0.8294.


### Ejercicio 2a:

El objetivo de este ejercicio es ver cuánto da el score si no se quiere hacer ninguna preparación de los datos.

Para esto, deben eliminarse las columnas que no tienen datos numéricos y las que tienen datos incompletos, ya que los algoritmos de aprendizaje automático no saben lidiar con ellos.

In [ ]:
# cargar los datos
import pandas as pd
train = pd.read_csv('datasets/titanic/train.csv')
train.info()

In [ ]:
del train['PassengerId']  # eliminar ids

# COMPLETAR: eliminar las columnas que no tienen números, y las que tienen datos incompletos.

In [ ]:
# entrenar la misma Random Forest, e imprimir el out-of-bag score
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(criterion='gini', 
                             n_estimators=700,
                             min_samples_split=10,
                             min_samples_leaf=1,
                             max_features='auto',
                             oob_score=True,
                             random_state=1,
                             n_jobs=-1)
rf.fit(train.iloc[:, 1:], train.iloc[:, 0])
print("%.4f" % rf.oob_score_)

### Ejercicio 2b:

En este ejercicio deberá hacer tres mejoras en la preparación de los datos:

- Completar la columna "Age" y usarla.
- Agregar una nueva columna con el largo del nombre.
- Convertir la columna "Sex" a valores numéricos y usarla.

Calcular el OOB score de cada una de las mejoras por separado.

**Responda**: ¿Cuál de las tres mejoras aporta más?

In [ ]:
# cargar nuevamente los datos
import pandas as pd
train = pd.read_csv('datasets/titanic/train.csv')
train.info()

In [ ]:
# COMPLETAR: completar con el valor medio los datos faltantes en el campo Age

In [ ]:
# COMPLETAR: agregar columna con el largo del nombre

In [ ]:
# COMPLETAR: poner 1 en lugar de 'female' y 0 en lugar de 'male'
# AYUDA: DEFINIR LA FUNCIÓN f Y HACER:
# train['Sex'] = train['Sex'].apply(f)

In [ ]:
# COMPLETAR: eliminar las columnas que deban ser eliminadas

In [ ]:
# EJECUTAR ESTO TRES VECES, UNA VEZ PARA CADA MEJORA.
# entrenar el Random Forest, e imprimir out-of-bag score
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(criterion='gini', 
                             n_estimators=700,
                             min_samples_split=10,
                             min_samples_leaf=1,
                             max_features='auto',
                             oob_score=True,
                             random_state=1,
                             n_jobs=-1)
rf.fit(train.iloc[:, 1:], train.iloc[:, 0])
print("%.4f" % rf.oob_score_)